# Basic ETL
---

## Goal
Consolidate the data from the two `json` files into a single more manageable storage type.

# Imports and setup
---

In [1]:
# automatically reloads modified libraries
%load_ext autoreload
%autoreload 2

# allows plots to be displayed inline
%matplotlib inline

# sharper figure for retina displays
%config InlineBackend.figure_format='retina'

In [2]:
# standard library
import html
import os
import re
import sys
import numpy as np
import pandas as pd

# third parties
import pandas as pd
import pyspark as ps

from pprint import pprint
from pyspark.sql.types import BooleanType
from pyspark.sql import functions as psf
from pyspark.sql.functions import udf, pandas_udf
from pyspark.sql.types import MapType, StringType, IntegerType, DataType, StructType, StructField

In [3]:
# create a spark session
spark = ps.sql.SparkSession.builder\
                            .master('local[*]')\
                            .appName('json_etl')\
                            .config('spark.executor.memory', '4gb')\
                            .config('spark.driver.maxResultSize', '32G')\
                            .getOrCreate()

# Metadata

Let's take a look at the metadata's schema.

In [11]:
# import the metadata into a 
meta = spark.read.json('../data/metadata.json.gz').cache()

In [12]:
meta.columns

['_corrupt_record',
 'asin',
 'brand',
 'categories',
 'description',
 'imUrl',
 'price',
 'related',
 'salesRank',
 'title']

First, I want to drop the corrupt records.

In [26]:
meta = meta.filter(meta['_corrupt_record'].isNull())

In [27]:
meta.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- related: struct (nullable = true)
 |    |-- also_bought: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- also_viewed: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bought_together: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- buy_after_viewing: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- salesRank: struct (nullable = true)
 |    |-- Appliances: long (nullable = true)
 |    |-- Arts, Crafts & Sewing: long (nullable = true)
 |    |-- Automotive: long

Now I want to do is to convert the zipped `json` file into something with better IO speed. A `parquet` format might be a good option, but we can see that under the sales rank field there a few illegal characters. This could be solved manually, but why not create an over engineered solution for a simple problem?

In [29]:
# create a new schema for the sales rank structure
pattern = re.compile(r' & |, ')
schema_str = meta.select('salesRank').schema.simpleString()
schema_str = re.findall('struct<salesRank:(.*)>', schema_str)[0]
new_schema = re.sub(pattern, '_', html.unescape(schema_str)).replace(' ', '_')

In [30]:
# replace white spaces, & and commas with underscores
meta = meta.withColumn('salesRank', meta['salesRank'].cast(new_schema))

In [31]:
meta.select('salesRank').printSchema()

root
 |-- salesRank: struct (nullable = true)
 |    |-- Appliances: long (nullable = true)
 |    |-- Arts_Crafts_Sewing: long (nullable = true)
 |    |-- Automotive: long (nullable = true)
 |    |-- Baby: long (nullable = true)
 |    |-- Beauty: long (nullable = true)
 |    |-- Books: long (nullable = true)
 |    |-- Camera_Photo: long (nullable = true)
 |    |-- Cell_Phones_Accessories: long (nullable = true)
 |    |-- Clothing: long (nullable = true)
 |    |-- Computers_Accessories: long (nullable = true)
 |    |-- Electronics: long (nullable = true)
 |    |-- Gift_Cards_Store: long (nullable = true)
 |    |-- Grocery_Gourmet_Food: long (nullable = true)
 |    |-- Health_Personal_Care: long (nullable = true)
 |    |-- Home_Kitchen: long (nullable = true)
 |    |-- Home_Improvement: long (nullable = true)
 |    |-- Industrial_Scientific: long (nullable = true)
 |    |-- Jewelry: long (nullable = true)
 |    |-- Kitchen_Dining: long (nullable = true)
 |    |-- Magazines: long (nullable

The categories' names got a bit more confusing, but now we will not run into problems when trying to convert the data frame to parquet.

In [32]:
meta.write.save('../data/metadata.parquet', format='parquet', mode='overwrite')

In [33]:
meta = spark.read.parquet('../data/metadata.parquet')

What about the categories? How many are available? What do they look like?

In [34]:
meta.select('categories').show()

+--------------------+
|          categories|
+--------------------+
|           [[Books]]|
|[[Movies & TV, Mo...|
|[[Clothing, Shoes...|
|[[Sports & Outdoo...|
|[[Sports & Outdoo...|
|[[Sports & Outdoo...|
|[[Movies & TV, Mo...|
|           [[Books]]|
|[[Sports & Outdoo...|
|           [[Books]]|
|           [[Books]]|
|           [[Books]]|
|           [[Books]]|
|           [[Books]]|
|           [[Books]]|
|           [[Books]]|
|           [[Books]]|
|           [[Books]]|
|[[Sports & Outdoo...|
|           [[Books]]|
+--------------------+
only showing top 20 rows



The categories seem to be strings separated by commas stored inside embedded lists. Let's try to solve this problem.

In [35]:
categories = psf.explode(meta['categories'])
exploded_meta = meta.withColumn('categories_exploded', categories)
exploded_meta.select('categories_exploded').show(5)

+--------------------+
| categories_exploded|
+--------------------+
|             [Books]|
|[Movies & TV, Mov...|
|[Clothing, Shoes ...|
|[Clothing, Shoes ...|
|[Sports & Outdoor...|
+--------------------+
only showing top 5 rows



We need to go deeper

In [36]:
exploded_meta.columns

['_corrupt_record',
 'asin',
 'brand',
 'categories',
 'description',
 'imUrl',
 'price',
 'related',
 'salesRank',
 'title',
 'categories_exploded']

In [37]:
categories = psf.explode(exploded_meta['categories_exploded'])
exploded_meta = exploded_meta.withColumn('categories_exploded', categories)
exploded_meta.select('categories_exploded').show(5)

+--------------------+
| categories_exploded|
+--------------------+
|               Books|
|         Movies & TV|
|              Movies|
|Clothing, Shoes &...|
|               Girls|
+--------------------+
only showing top 5 rows



Aha! Now is only a matter of splitting the categories.

In [38]:
categories = psf.explode(exploded_meta['categories_exploded'])
exploded_meta = exploded_meta.withColumn('category', psf.explode(psf.split('categories_exploded', ', ')))
exploded_meta.select('category').show(5)

+---------------+
|       category|
+---------------+
|          Books|
|    Movies & TV|
|         Movies|
|       Clothing|
|Shoes & Jewelry|
+---------------+
only showing top 5 rows



In [39]:
exploded_meta.select('category').distinct().show()

+--------------------+
|            category|
+--------------------+
|    Personal Finance|
|    Stress Reduction|
|      Micro SD Cards|
|      Pegged Puzzles|
|         Note Taking|
|Hand & Arm Protec...|
|                Goth|
|    Ballets & Dances|
|            Ska Punk|
|        Bread Knives|
|  Skullies & Beanies|
|Bathroom Accessories|
|         Knife Cases|
|      Serving Dishes|
|           Envelopes|
|              Easter|
|             Septets|
|            Ranchera|
|Portable & Novelt...|
|          Aida Cloth|
+--------------------+
only showing top 20 rows



In [40]:
exploded_meta.select('category').distinct().count()

18360

Hmmm... using 18k categories doesn't sound very promising...

In [41]:
exploded_meta.unpersist();

Let's take a look a `salesRank` instead

In [42]:
salesrank_samples = meta.select('salesRank').take(20)

In [43]:
salesrank_dicts = [sample['salesRank'].asDict() for sample in salesrank_samples if sample['salesRank']]
salesrank_df = pd.DataFrame(salesrank_dicts)
# count the number of non null values in each row
salesrank_df.count(axis=1)

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
dtype: int64

It looks like each `salesRank` contains only one not null value, but just to be sure...

In [44]:
@udf
def cat_counts(struct):
    '''
    Simple function used to could the number or non null values
    in each `salesRank` field.
    '''
    if struct:
        struct_values = struct.asDict().values()
        cats = 0
        for value in struct_values:
            if value:
                cats += 1
        return cats

In [45]:
meta.select(cat_counts('salesRank').alias('n_cats')).groupby('n_cats').count().show()

+------+-------+
|n_cats|  count|
+------+-------+
|     0|  23623|
|  null|2228167|
|     1|6151598|
+------+-------+



Yeah, each observation has at most one category. Good.

In [47]:
struct_sample = meta.select('salesRank').take(1)[0]
cats = struct_sample['salesRank'].__fields__
print(f'Number of distinct categories: {len(cats)}')
cats

Number of distinct categories: 33


['Appliances',
 'Arts_Crafts_Sewing',
 'Automotive',
 'Baby',
 'Beauty',
 'Books',
 'Camera_Photo',
 'Cell_Phones_Accessories',
 'Clothing',
 'Computers_Accessories',
 'Electronics',
 'Gift_Cards_Store',
 'Grocery_Gourmet_Food',
 'Health_Personal_Care',
 'Home_Kitchen',
 'Home_Improvement',
 'Industrial_Scientific',
 'Jewelry',
 'Kitchen_Dining',
 'Magazines',
 'Movies_TV',
 'Music',
 'Musical_Instruments',
 'Office_Products',
 'Patio_Lawn_Garden',
 'Pet_Supplies',
 'Prime_Pantry',
 'Shoes',
 'Software',
 'Sports_Outdoors',
 'Toys_Games',
 'Video_Games',
 'Watches']

Aahhh, much more manageable. From now on, this will be our categories.

In [48]:
schema = StructType([
    StructField('category', StringType()),
    StructField('rank', IntegerType())
])

@udf(schema)
def get_catrank(struct):
    '''
    Returns the category with non null value if it exists.
    '''
    if struct:
        struct_items = list(struct.asDict().items())
        for cat, rank in struct_items:
            if rank:
                cat_field = StructField('category', StringType(), )
                return {'category': cat, 'rank': rank}

In [49]:
cat_ranks = get_catrank(meta['salesRank'])
category = cat_ranks.getField('category').alias('category')
sales_rank = cat_ranks.getField('rank').alias('sales_rank')

meta = meta.withColumn('category', category)
meta = meta.withColumn('sales_rank', sales_rank)
meta.columns

['_corrupt_record',
 'asin',
 'brand',
 'categories',
 'description',
 'imUrl',
 'price',
 'related',
 'salesRank',
 'title',
 'category',
 'sales_rank']

Now it would be a good idea to remove unnecessary columns such as `imUrl` and `brand`. I will leave in `price` just out of curiosity and `asin` is necessary to join this data frame with the reviews one latter.

In [50]:
keep_cols = {
    'asin',
    'category',
    'price',
    'sales_rank'
}

meta = meta.select(*keep_cols)

In [51]:
meta.write.parquet('../data/metadata_clean.parquet', mode='overwrite')

# Reviews

In [7]:
reviews = spark.read.json('../data/amazon_reviews.json.gz')

In [50]:
reviews.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



We already have `unixReviewTime` so the `reviewTime` column is redundant. We also don't care about the `reviewerID` or `reviewerName`, so they can be dropped as well.

In [9]:
reviews = reviews.drop('reviewTime', 'reviewerName', 'reviewerID')

In [10]:
reviews.show(3)

+----------+-------+-------+--------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText|             summary|unixReviewTime|
+----------+-------+-------+--------------------+--------------------+--------------+
|0000013714| [0, 0]|    4.0|We use this type ...|         Nice Hymnal|    1386028800|
|0000013714| [2, 3]|    5.0|I bought this for...|Heavenly Highway ...|    1252800000|
|0000013714| [0, 0]|    5.0|This is a large s...|   Awesome Hymn Book|    1362787200|
+----------+-------+-------+--------------------+--------------------+--------------+
only showing top 3 rows



Now, the helpful column is populated by arrays of length 2, where the first element corresponds to the number of positive votes given to the review and while the second element represents the total number of votes (positive + negative). In order to make the data more easily accessible, it is a good idea to extract to explode the array and create two columns out of it, `positiveVotes` and `totalVotes`.

In [11]:
positive_votes = reviews['helpful'].getItem(0)
total_votes = reviews['helpful'].getItem(1)

reviews = reviews.withColumn('positiveVotes', positive_votes)
reviews = reviews.withColumn('totalVotes', total_votes)

reviews.show(5)

+----------+-------+-------+--------------------+--------------------+--------------+-------------+----------+
|      asin|helpful|overall|          reviewText|             summary|unixReviewTime|positiveVotes|totalVotes|
+----------+-------+-------+--------------------+--------------------+--------------+-------------+----------+
|0000013714| [0, 0]|    4.0|We use this type ...|         Nice Hymnal|    1386028800|            0|         0|
|0000013714| [2, 3]|    5.0|I bought this for...|Heavenly Highway ...|    1252800000|            2|         3|
|0000013714| [0, 0]|    5.0|This is a large s...|   Awesome Hymn Book|    1362787200|            0|         0|
|0000013714| [0, 0]|    5.0|We use this hymn ...|Hand Clapping Toe...|    1325462400|            0|         0|
|0000013714| [0, 0]|    3.0|One review advise...|          Misleading|    1376092800|            0|         0|
+----------+-------+-------+--------------------+--------------------+--------------+-------------+----------+
o

Nice! Now we can drop the `helpful` column.

In [12]:
reviews = reviews.drop('helpful')

Another thing that I would like to check is whether or not `overall` contains "half-stars". If not, it makes more sense to store the values as integers instead of doubles.

In [13]:
reviews.select('overall').distinct().show()

+-------+
|overall|
+-------+
|    1.0|
|    4.0|
|    3.0|
|    2.0|
|    5.0|
+-------+



Integers it is!

In [14]:
integer_overall = reviews['overall'].cast('integer')
reviews = reviews.withColumn('overall', integer_overall)

This is the final `reviews` data frame.

In [15]:
reviews.show()

+----------+-------+--------------------+--------------------+--------------+-------------+----------+
|      asin|overall|          reviewText|             summary|unixReviewTime|positiveVotes|totalVotes|
+----------+-------+--------------------+--------------------+--------------+-------------+----------+
|0000013714|      4|We use this type ...|         Nice Hymnal|    1386028800|            0|         0|
|0000013714|      5|I bought this for...|Heavenly Highway ...|    1252800000|            2|         3|
|0000013714|      5|This is a large s...|   Awesome Hymn Book|    1362787200|            0|         0|
|0000013714|      5|We use this hymn ...|Hand Clapping Toe...|    1325462400|            0|         0|
|0000013714|      3|One review advise...|          Misleading|    1376092800|            0|         0|
|0000029831|      4|This work bears d...|Logical follow-up...|    1393286400|            3|         3|
|0000029831|      5|You may laugh, bu...|The superior clea...|    1393200

In [17]:
reviews.write.save('../data/reviews_clean.parquet', format='parquet', mode='overwrite')

Now it is time to join the two data frames into one.

# Final data frame

In [4]:
meta = spark.read.parquet('../data/metadata_clean.parquet')
reviews = spark.read.parquet('../data/reviews_clean.parquet')

In [5]:
final_df = reviews.join(meta, on='asin', how='inner')

In [6]:
final_df.write.parquet('../data/amazon.parquet', mode='overwrite')

In [8]:
final_df = spark.read.parquet('../data/amazon.parquet')

In [12]:
final_df.drop('asin').show()

+-------+--------------------+--------------------+--------------+-------------+----------+-----+--------+----------+
|overall|          reviewText|             summary|unixReviewTime|positiveVotes|totalVotes|price|category|sales_rank|
+-------+--------------------+--------------------+--------------+-------------+----------+-----+--------+----------+
|      5|This book hits th...|        An easy read|    1383091200|            0|         0| 6.64|   Books|   1127575|
|      4|Very good book de...|Interesting pictu...|    1352592000|            0|         0| 6.64|   Books|   1127575|
|      4|Some of Agatha Ch...|An engaging memoi...|    1282608000|            4|         4| 6.64|   Books|   1127575|
|      5|This inspiring au...|An Autobiography,...|    1390780800|            0|         0| 6.64|   Books|   1127575|
|      4|Sure reading Agat...|"Instead  I went ...|    1312156800|            3|         4| 6.64|   Books|   1127575|
|      4|I was a bit hesit...|Personally and cu...|    1

Beautiful! That is it for ETL!